In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [6]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 164 ( 0.5%) Users that have less than 1 test interactions


In [7]:
from libs.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from libs.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

In [21]:
def objective_function_P3_beta(optuna_trial):

    
                          
        
    recommender_instance = RP3betaRecommender(URM_train)
    recommender_instance.fit(topK = optuna_trial.suggest_int("topK", 2, 50),
                             alpha = optuna_trial.suggest_float("alpha",0.1, 0.9),
                             beta = optuna_trial.suggest_float("beta",0.0, 0.99),
                             normalize_similarity= True,
                             implicit=False,
                            )
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)    
    return eval_res["MAP"][10]

In [12]:
def objective_function_P3_alpha(optuna_trial):

    
                          
        
    recommender_instance = P3alphaRecommender(URM_train)
    recommender_instance.fit(topK = optuna_trial.suggest_int("topK", 2, 400),
                             alpha=  optuna_trial.suggest_float("alpha",0.35, 0.39,log=False),
                             normalize_similarity= True
                            )
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)    
    return eval_res["MAP"][10]

In [19]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame()
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        
        self.results_df = self.results_df._append(hyperparam_dict, ignore_index=True)

In [22]:
optuna_study_beta = op.create_study(direction="maximize")
        
save_results_beta = SaveResults()
        
optuna_study_beta.optimize(objective_function_P3_beta,
                      callbacks=[save_results_beta],
                      n_trials = 300)

[I 2024-12-09 12:50:45,950] A new study created in memory with name: no-name-70334c15-c38c-4fcc-8788-8db61abec324


RP3betaRecommender: Similarity column 38121 (100.0%), 4168.22 column/sec. Elapsed time 9.15 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 9.21 sec. Users per second: 3864


[I 2024-12-09 12:51:04,777] Trial 0 finished with value: 0.05272370791901432 and parameters: {'topK': 19, 'alpha': 0.12530212218934408, 'beta': 0.4783219709680538}. Best is trial 0 with value: 0.05272370791901432.


RP3betaRecommender: Similarity column 38121 (100.0%), 4213.86 column/sec. Elapsed time 9.05 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 8.62 sec. Users per second: 4124


[I 2024-12-09 12:51:22,699] Trial 1 finished with value: 0.047363425888739104 and parameters: {'topK': 4, 'alpha': 0.5466211536511135, 'beta': 0.022527176686367594}. Best is trial 0 with value: 0.05272370791901432.


...

[I 2024-12-09 15:00:51,440] Trial 297 finished with value: 0.055468339196690794 and parameters: {'topK': 14, 'alpha': 0.35969554696939554, 'beta': 0.16501812083151052}. Best is trial 235 with value: 0.05558903002896657.


RP3betaRecommender: Similarity column 38121 (100.0%), 4225.03 column/sec. Elapsed time 9.02 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 8.73 sec. Users per second: 4073


[I 2024-12-09 15:01:09,540] Trial 298 finished with value: 0.055199402196124944 and parameters: {'topK': 10, 'alpha': 0.32011867736318506, 'beta': 0.19678543071053398}. Best is trial 235 with value: 0.05558903002896657.


RP3betaRecommender: Similarity column 38121 (100.0%), 3972.71 column/sec. Elapsed time 9.60 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 8.84 sec. Users per second: 4022


[I 2024-12-09 15:01:28,324] Trial 299 finished with value: 0.05529931469195302 and parameters: {'topK': 13, 'alpha': 0.3381390676689048, 'beta': 0.11452356481427528}. Best is trial 235 with value: 0.05558903002896657.


In [23]:
save_results_beta.results_df.to_csv("result_experiments/RP3beta/results_optuna_50.csv")

In [15]:
optuna_study_alpha = op.create_study(direction="maximize")
        
save_results_alpha = SaveResults()
        
optuna_study_alpha.optimize(objective_function_P3_alpha,
                      callbacks=[save_results_alpha],
                      n_trials = 300)

[I 2024-12-09 07:40:21,634] A new study created in memory with name: no-name-190db610-7b6e-4d85-a877-2611764cbb27


P3alphaRecommender: Similarity column 38121 (100.0%), 4703.27 column/sec. Elapsed time 8.11 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 9.02 sec. Users per second: 3946


[I 2024-12-09 07:40:39,405] Trial 0 finished with value: 0.05047467220517467 and parameters: {'topK': 51, 'alpha': 0.3774241480510347}. Best is trial 0 with value: 0.05047467220517467.


P3alphaRecommender: Similarity column 38121 (100.0%), 4690.72 column/sec. Elapsed time 8.13 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 9.03 sec. Users per second: 3941


[I 2024-12-09 07:40:57,216] Trial 1 finished with value: 0.05040746556503141 and parameters: {'topK': 52, 'alpha': 0.3846647742101469}. Best is trial 0 with value: 0.05047467220517467.


...

[I 2024-12-09 09:11:02,485] Trial 297 finished with value: 0.05193420476065364 and parameters: {'topK': 35, 'alpha': 0.35649162064736145}. Best is trial 101 with value: 0.054204765117559894.


P3alphaRecommender: Similarity column 38121 (100.0%), 5014.79 column/sec. Elapsed time 7.60 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 8.50 sec. Users per second: 4187


[I 2024-12-09 09:11:18,922] Trial 298 finished with value: 0.05394699259627957 and parameters: {'topK': 16, 'alpha': 0.3549334113308688}. Best is trial 101 with value: 0.054204765117559894.


P3alphaRecommender: Similarity column 38121 (100.0%), 5082.31 column/sec. Elapsed time 7.50 sec
EvaluatorHoldout: Processed 35572 (100.0%) in 8.31 sec. Users per second: 4279


[I 2024-12-09 09:11:35,008] Trial 299 finished with value: 0.05328089553224295 and parameters: {'topK': 8, 'alpha': 0.3511059201165217}. Best is trial 101 with value: 0.054204765117559894.


In [17]:
save_results_alpha.results_df.to_csv("result_experiments/RP3alpha/results_optuna.csv")